In [1]:
import hourse_price_preprocessor
from stacking import MyStackingRegressor
import numpy as np
from pandas import DataFrame

from sklearn.model_selection import cross_val_score


In [2]:
import warnings
import copy
warnings.filterwarnings('ignore')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
trainset_file = "data/house_price/train.csv"
test_file = "data/house_price/test.csv"

X_train, X_test, y_train, test_id_idx = \
            hourse_price_preprocessor.get_train_test_split_dataset(trainset_file,test_file)

In [5]:
from xgboost import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor

In [6]:
estimator1 = XGBRegressor(max_depth=3, learning_rate=0.5, n_estimators=50)
estimator2 = LGBMRegressor(max_depth=2, learning_rate=0.5, n_estimators=50)
estimator3 = RandomForestRegressor(n_estimators=500, max_depth=3)
estimator4 = GaussianNB()
estimator5 = LinearSVR()
estimator6 = MLPRegressor(hidden_layer_sizes=(512,256, 32))

In [12]:
base_estimators = [estimator1, estimator2, estimator3, estimator4, estimator5, estimator6]

In [13]:
meta_estimator = XGBRegressor(max_depth=3, learning_rate=0.5, n_estimators=50)

In [14]:
stack_regressor = MyStackingRegressor(meta_estimator, base_estimators)

In [15]:
stack_regressor.fit(X_train, y_train)

[16:07:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MyStackingRegressor(base_estimators=[XGBRegressor(base_score=0.5,
                                                  booster='gbtree',
                                                  colsample_bylevel=1,
                                                  colsample_bynode=1,
                                                  colsample_bytree=1, gamma=0,
                                                  importance_type='gain',
                                                  learning_rate=0.5,
                                                  max_delta_step=0, max_depth=3,
                                                  min_child_weight=1,
                                                  missing=None, n_estimators=50,
                                                  n_jobs=1, nthread=None,
                                                  objective='reg:linear',
                                                  random_state=0, reg_alpha=0,
                                            

In [19]:
cross_val_score(stack_regressor, X_train, y_train, scoring="neg_mean_squared_error" , cv=5).mean()

[16:08:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:08:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:08:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:08:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:08:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:08:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:08:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:09:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:09:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

-1319395967.5654697

In [20]:

for estimator in base_estimators:
    stack_regressor = MyStackingRegressor(copy.copy(estimator), base_estimators)
    stack_regressor.fit(X_train, y_train)
    result = cross_val_score(stack_regressor, X_train, y_train, scoring="neg_mean_squared_error" , cv=5).mean()
    print(result)

[16:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:09:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:09:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:09:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:09:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:10:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:10:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:10:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:10:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [21]:
for estimator in base_estimators:
    general_estimator = copy.copy(estimator)
    general_estimator.fit(X_train, y_train)
    result = cross_val_score(general_estimator, X_train, y_train, scoring="neg_mean_squared_error" , cv=5).mean()
    del general_estimator
    print(result)

[16:18:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-940351621.2845335
-1141626511.0051265
-1778803230.4692063
-2113784463.8692105
-36887497534.321045
-1198721564.439732


In [22]:
general_estimator = copy.copy(base_estimators[0])
general_estimator.fit(X_train, y_train)
general_estimator.predict(X_test)

[16:20:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([128621.04 , 155807.97 , 197459.06 , ..., 130143.836, 114837.03 ,
       260349.64 ], dtype=float32)

In [23]:
def make_submission(estimator, X_test, test_id_idx, filename, column_names):
    id_value=test_id_idx
    sales_price = estimator.predict(X_test)
    result = np.vstack([id_value, sales_price]).T
    submission_df =DataFrame(result, columns=column_names).set_index(column_names[0])
    submission_df.index = submission_df.index.astype(int)
    submission_df.to_csv(filename)


In [24]:
# make_submission(general_estimator, X_test, test_id_idx)

TypeError: make_submission() missing 2 required positional arguments: 'filename' and 'column_names'

In [25]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f house_price_result_search.csv -m "Message"

'kaggle'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [26]:
general_estimator = copy.copy(base_estimators[2])
stack_regressor = MyStackingRegressor(general_estimator, base_estimators)
stack_regressor.fit(X_train, y_train)

[16:26:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MyStackingRegressor(base_estimators=[XGBRegressor(base_score=0.5,
                                                  booster='gbtree',
                                                  colsample_bylevel=1,
                                                  colsample_bynode=1,
                                                  colsample_bytree=1, gamma=0,
                                                  importance_type='gain',
                                                  learning_rate=0.5,
                                                  max_delta_step=0, max_depth=3,
                                                  min_child_weight=1,
                                                  missing=None, n_estimators=50,
                                                  n_jobs=1, nthread=None,
                                                  objective='reg:linear',
                                                  random_state=0, reg_alpha=0,
                                            

In [ ]:
make_submission(stack_regressor, X_test, test_id_idx)

In [ ]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f house_price_result_search.csv -m "Message"

## Classifier problems

In [27]:
from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from stacking import MyStackingClassifier

In [28]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [29]:
estimator1 = XGBClassifier(max_depth=3, learning_rate=0.5, n_estimators=50)
estimator2 = LGBMClassifier(max_depth=2, learning_rate=0.5, n_estimators=50)
estimator3 = RandomForestClassifier(n_estimators=500, max_depth=3)
estimator4 = LinearSVC()
estimator5 = MLPClassifier(hidden_layer_sizes=(512,256, 32))

In [30]:
base_estimators = [estimator1, estimator2, estimator3, estimator4, estimator5]

In [31]:
for estimator in base_estimators:
    stack_classifier = MyStackingClassifier(copy.copy(estimator), base_estimators)
    result = cross_val_score(stack_classifier, X, y, scoring="accuracy" , cv=5).mean()
    print(result)

0.8200279311877103
0.8177997841680948
0.8346854567383991
0.8369199517552213
0.8312511902494762


In [32]:
for estimator in base_estimators:
    general_estimator = copy.copy(estimator)
    result = cross_val_score(general_estimator, X, y, scoring="accuracy" , cv=5).mean()
    del general_estimator
    print(result)

0.840290738272075
0.8346981527328128
0.8076556846314988
0.8313083222243381
0.8133307941344506


In [33]:
X_test = np.load("tatanic_test.npy")

In [34]:
idx = list(range(892, 1310))

In [35]:
general_estimator = copy.copy(base_estimators[5])
general_estimator.fit(X, y)

IndexError: list index out of range

In [ ]:
column_names = ["PassengerId", "Survived"]

In [ ]:
def make_submission(estimator, X_test, test_id_idx, filename, column_names):
    id_value=test_id_idx
    sales_price = np.array(estimator.predict(X_test), dtype=int)
    result = np.vstack([id_value, sales_price]).T
    submission_df =DataFrame(result, columns=column_names).set_index(column_names[0])
    submission_df.index = submission_df.index.astype(int)
    submission_df.to_csv(filename)


In [ ]:
make_submission(general_estimator, X_test, idx, "titanic_submssion.csv", column_names)

In [ ]:
!kaggle competitions submit -c titanic -f titanic_submssion.csv -m "Message"

In [ ]:
general_estimator = copy.copy(base_estimators[4])
stack_estimator = MyStackingClassifier(general_estimator, base_estimators)
result = stack_estimator.fit(X, y)
make_submission(result, X_test, idx, "titanic_submssion.csv", column_names)

In [ ]:
!kaggle competitions submit -c titanic -f titanic_submssion.csv -m "Message"